In [5]:
import pandas as pd
import mysql.connector

# Kết nối đến cơ sở dữ liệu MySQL
def connect_to_mysql(host, user, password, database, port):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Lỗi kết nối đến MySQL: {err}")
        return None

# Đọc dữ liệu từ file CSV và đẩy vào MySQL
def push_csv_to_mysql(csv_file, table_name, conn):
    try:
        df = pd.read_csv(csv_file)
        
        # Thay thế các giá trị NaN bằng chuỗi trống
        df = df.fillna('')

        cursor = conn.cursor()
        
        # Xóa bảng nếu tồn tại
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
        
        # Tạo bảng mới với các cột từ file CSV
        create_table_query = f"CREATE TABLE {table_name} ("
        for column in df.columns:
            create_table_query += f"{column} VARCHAR(255),"
        create_table_query = create_table_query[:-1] + ")"
        cursor.execute(create_table_query)
        
        # Đẩy dữ liệu vào bảng
        for row in df.itertuples(index=False):
            placeholders = ', '.join(['%s'] * len(row))
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, tuple(row))
        
        conn.commit()
        cursor.close()
        print("Đã đẩy dữ liệu từ CSV vào MySQL thành công.")
    except Exception as e:
        print("Lỗi khi đẩy dữ liệu từ CSV vào MySQL:", e)
        conn.rollback()
    finally:
        if conn.is_connected():
            conn.close()

# Thay các thông tin kết nối và tên file CSV tại đây
host = "monorail.proxy.rlwy.net"
user = "root"
password = "xIbazQgoTOmqlVFBFVpELtNnscUXRTfq"
database = "railway"
port = 45681
#Thay tên file csv
csv_file = 'product.csv'
#Thay tên table 
table_name = 'san_pham'

# Kết nối đến MySQL
conn = connect_to_mysql(host, user, password, database, port)
if conn:
    # Đẩy dữ liệu từ CSV vào MySQL
    push_csv_to_mysql(csv_file, table_name, conn)
else:
    print("Không thể kết nối đến MySQL.")


Đã đẩy dữ liệu từ CSV vào MySQL thành công.
